## Deep Learning Steps - PoseNet
Both scripts aim to detect exercise segments based on PoseNet data. They share several core steps:

1. **Data Loading & Preprocessing**  
   - Read full-pose CSV files and corresponding trimmed CSVs.  
   - Smooth sequences via a moving-average window and compute deltas.  
   - Derive labels indicating exercise frames vs. non-exercise frames.

2. **Train/Test Split**  
   - Group-based splitting ensures entire videos fall into either train or test.

3. **Feature Scaling**  
   - A `StandardScaler` is fit on training data only, then applied to both sets.

4. **Model Definition & Training**  
   - The first script trains an MLP classification model (using scikeras + GridSearchCV).  
   - The second script handles two modes:  
     - “frame” mode with a weighted-BCE loss per frame.  
     - “boundary” mode using MSE on the normalized start/end positions.

5. **Evaluation**  
   - Compute frame-level metrics like precision, recall, F1, and boundary errors.  
   - In the second script, boundary-mode evaluation checks how close predicted start/end match the ground-truth.

###  Parameter Variations
1. **First Script (postnetcutting.py)**  
   - Tries a grid of MLP hyperparameters (hidden_units=256, hidden_layers=12, dropout_rate=0.30).  
   - Uses up to 50 epochs and a batch size of 64.

2. **Second Script (postnetcutting2.py)**  
   - Selects number of layers (default 12) and hidden units (128).  
   - Adjusts mode (“frame” / “boundary”) for loss function choice.  
   - Configures distinct hyperparameters (dropout=0, LR=1e-4, batch size=256, epochs=50).

### Results
After training, the aggregate boundary error was:

──────── Aggregate boundary error ────────

Δstart mean +5.33 ± 25.88 median |Δ| 12.0 frames

Δend mean -3.28 ± 21.89 median |Δ| 8.5 frames

This indicates that on average, the model predicts start frames 5.33 frames later and end frames 3.28 frames earlier than the ground truth. The high standard deviations (25.88 and 21.89 frames) suggest considerable variability in predictions. The median absolute errors (12.0 for start, 8.5 for end) provide a more robust measure of typical error, less affected by outliers.


## Deep Learning Steps - Kinect

Regarding the kinectcutting.py, the following steps were taken:

1. Data Loading and Preprocessing

* Loads matched CSV files from uncut and preprocessed directories.
* Applies data augmentation: mirroring and rotation (±15 degrees).
* Creates sliding windows of 11 frames for input features.

2. Data Splitting

* Uses GroupShuffleSplit to ensure entire sequences remain in either train or test set.
* 80% train, 20% test split.

3. Feature Scaling

* Applies StandardScaler to normalize input features.

4. Model Architecture

* Implements a sequential model with multiple dense layers.
* Uses ReLU activation for hidden layers and sigmoid for output.
* Compiles with binary cross-entropy loss and Adam optimizer.

5. Hyperparameter Tuning

* Utilizes GridSearchCV for hyperparameter optimization.
* Searches over number of layers, units, learning rate, batch size, and epochs.

6. Model Training

* Trains the model using the best hyperparameters from GridSearchCV.
* Implements early stopping to prevent overfitting.

7. Evaluation

* Evaluates on test set for loss, accuracy, precision, and recall.
* Performs detailed boundary error analysis per video.

### Parameter Variations
The script explores the following hyperparameter space:

* Number of layers: [12]
* Number of units per layer: [128]
* Learning rate: [1e-4]
* Batch size: [128]
* Number of epochs: [50]
* Early stopping is implemented with a patience of 5 epochs, monitoring validation loss.

### Result

───────── Aggregate boundary error ─────────

Δstart  mean  +2.83 ± 19.14   median |Δ| 5.0 frames

Δend    mean  +1.22 ± 16.01   median |Δ| 3.0 frames

### These results indicate:

Start frames are predicted, on average, 2.83 frames later than actual.
End frames are predicted, on average, 1.22 frames later than actual.
The median absolute errors (5.0 for start, 3.0 for end) suggest relatively good accuracy in boundary detection.
Standard deviations (19.14 for start, 16.01 for end) indicate some variability in predictions.
Overall, the model shows promising performance in detecting exercise segment boundaries, with slightly better accuracy for end frames compared to start frames.




## Software development

# Frame Trimmer Implementation Progress Report

## Project Overview

We have successfully implemented the Frame Trimmer component for the ML Prediction Dashboard that allows users to:

1. Upload CSV files containing skeletal keypoint data
2. Send the CSV to the backend for frame trimming
3. Visualize both the original and trimmed data in a side-by-side 3D comparison
4. Clearly see which frames were kept vs. removed during the trimming process

The Frame Trimmer has been integrated as a new tab in the existing dashboard alongside the "Predictions" and "PoseNet" tabs.

## Implementation Details

### Frontend Components

1. **FrameTrimmer.jsx**
   - Main component that handles file upload, processing, and visualization
   - Integrated with the existing SkeletonContext and SkeletonRenderer components
   - Includes a visual timeline showing which frames were kept vs. removed
   - Implements custom playback controls that handle all frames

2. **Custom Animation and Controls**
   - Replaced the standard AnimationManager with a custom implementation
   - Created custom controls to properly handle the original frame count
   - Fixed issues with playback that previously stopped at the end of trimmed data

3. **Visual Frame Timeline**
   - Implemented a series of colored div elements to create a timeline
   - Green segments show frames that were kept in the trimmed dataset
   - Red segments show frames that were removed
   - Interactive elements allow users to click to jump to specific frames
   - Optimized for performance with large datasets by implementing frame sampling

### Backend Endpoint

1. **`/trim-frames` Endpoint**
   - Added a new POST endpoint to the FastAPI application
   - Accepts a CSV file and returns a trimmed version
   - Currently returns a pre-trimmed output.csv file using our model(manually, without the pipeline) for testing
   - Returns the CSV content directly as text

## Technical Challenges and Interim Solutions

### Backend Processing Issues

1. **Temporary Manual Workflow**
   - **IMPORTANT NOTE:** Due to time constraints and the need to deliver a functional demonstration, we've established a temporary manual workflow:
     1. Manually preprocess CSV files using the working script
     2. Upload the processed files to the test folder of the backend
     3. Compare the uploaded uncut data with the manually prepared trimmed data(uncut.csv can also be found in the backend folder)
   - This approach allows us to showcase the functionality while we address the backend processing issues



## User Experience Improvements

1. **Intuitive Visualization**
   - Color-coded timeline provides immediate visual feedback on trimmed frames
   - Clear indicators show which frames were kept vs. removed
   - Empty placeholder for removed frames clearly communicates when a frame was trimmed

2. **Feedback and Status**
   - Added informative badges showing frame counts and percentage removed
   - Included status indicators to show current frame in both datasets
   - Added an informative badge explaining the color coding

3. **Responsive Controls**
   - Implemented smooth playback controls
   - Added interactive timeline with click-to-seek functionality
   - Ensured responsive design for different screen sizes

## Technical Architecture

The Frame Trimmer follows a clean architecture pattern:

1. **Data Flow**
   - User uploads CSV file → Frontend parses and displays original data
   - CSV is sent to backend → Backend processes and returns trimmed data
   - Frontend parses trimmed data → Both datasets visualized side by side

2. **State Management**
   - Uses React useState for component-level state
   - Leverages SkeletonContext for shared visualization state
   - Maintains mapping between original and trimmed frames

3. **Optimization Techniques**
   - Memoization of expensive calculations with useMemo
   - Callback optimization with useCallback
   - Conditional rendering to reduce DOM elements
   - Sampling for large datasets to maintain performance

## Conclusion

Despite the backend processing challenges, the Frame Trimmer component successfully integrates with the existing ML Prediction Dashboard, providing a powerful tool for visualizing frame trimming results. The current manual workflow is a temporary solution that allows us to demonstrate the functionality while we work on fixing the backend issues.